## Plotting the qualitative data

First use calibration from Ga and As to make a table to see if the peaks deviate from the expected values.

Then plot the qualitative data of each sample place.

In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import hyperspy.api as hs
from scipy.signal import find_peaks

# utils and helper_files
from utils.get_raw_data import get_array_and_names
from helper_files.normalizations import normalize_high_peak
from helper_files.peak_ratios import channel_to_kev, make_fit, n_gaussians_and_m_order_bg
from helper_files.calibration import calibrate_channel_width_two_peaks

In [2]:
ga, ga_names = get_array_and_names('GaAs')
mo, mo_names = get_array_and_names('Mo')
nw, nw_names = get_array_and_names('NW')
cu, cu_names = get_array_and_names('Cu') # mostly C
si, si_names = get_array_and_names('Si')
fe, fe_names = get_array_and_names('Fe') # it is Al not Fe

In [3]:
def calibrate_spectrum(spectrum, peaks_keV, initial_dispersion=0.01, initial_offset=20, deg=12, plot_fit=False, prominence=0.01):
    """
    Calibrate a spectrum using two peak positions.
    Finding the peaks automatically.
    Making a fit model with n gaussians and <deg> order background.
    Finding the closest peaks to the given peaks_keV.
    Then returning the dispersion and offset.
    """
    channels = np.arange(len(spectrum))
    # find peaks
    peaks_channel_est, _ = find_peaks(spectrum, prominence=0.01)
    peaks_keV_est = channel_to_kev(peaks_channel_est, dispersion=initial_dispersion, offset=initial_offset)

    # find the peaks that are closest to the peaks_keV
    peaks_to_fit_index = []
    for peak in peaks_keV:
        peaks_to_fit_index.append(np.argmin(np.abs(peaks_keV_est - peak)))

    # fit with make_fit
    # make_fit(array, x, deg=12, prominence=0.01, pixel_removal=50, offset=20) >>> fit_vals, covar
    fit_vals, _ = make_fit(spectrum, channels, deg=deg, prominence=prominence)


    fit_peaks = fit_vals[deg + 2::3]
    dispersion, offset = calibrate_channel_width_two_peaks(fit_peaks[peaks_to_fit_index], peaks_keV)
    fit_peaks_keV = channel_to_kev(fit_peaks, dispersion=dispersion, offset=offset)
    keV = channel_to_kev(channels, dispersion=dispersion, offset=offset)
    if plot_fit:
        fit = n_gaussians_and_m_order_bg(channels, deg, *fit_vals)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=keV, y=spectrum, name='spectrum', mode='markers'))
        fig.add_trace(go.Scatter(x=keV, y=fit, name='fit'))
        fig.add_vline(x=fit_peaks_keV[peaks_to_fit_index[0]], line_width=2, line_dash="dash")
        fig.add_vline(x=fit_peaks_keV[peaks_to_fit_index[1]], line_width=2, line_dash="dash")
        fig.update_layout(title='Calibration of the spectrum', xaxis_title='Energy [keV]', yaxis_title='Relative intensity')
        fig.show()
    
    return dispersion, offset

def find_nearest(array, value, max_deviation=0.05):
    """
    Finds the index of the value in array that is closest to value.
    If the deviation is larger than max_deviation in percent, returns None.

    Parameters
    ----------
    array : array of floats
        peak array
    value : float
        float to find the closest value to
    max_deviation : float, optional
        max deviation in percent, by default 0.05

    Returns
    -------
    index : int or None
        Index of the closest value in array to value. None if the deviation is larger than max_deviation.
    """
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    # check if the deviation is larger than max_deviation in percent
    if np.abs(array[idx] - value) / value > max_deviation:
        print(f'Warning: {array[idx]} is not within {max_deviation * 100}% of {value}')
        return idx
    return idx

def peak_deviation(peaks_energy, peaks_fit):
    """
    Finds the corresponding peaks in peaks_fit to peaks_energy,
    and returns the deviation in percent.

    peaks_energy is the array of the peaks in keV, in their theoretical position.
    """
    peaks_energy = np.array(peaks_energy)
    peaks_fit_selected = []
    for p in peaks_energy:
        index = find_nearest(peaks_fit, p)
        if index is not None:
            peaks_fit_selected.append(peaks_fit[index])

    peaks_fit_selected = np.array(peaks_fit_selected)
    # deviation = np.abs(peaks_fit_selected - peaks_energy) / peaks_energy * 100
    # deviation = (peaks_fit_selected - peaks_energy) / peaks_energy * 100 # this is in percent
    deviation = (peaks_fit_selected - peaks_energy) * 1000 # deviation in eV
    deviation_percent = (peaks_fit_selected - peaks_energy) / peaks_energy * 100

    return deviation, peaks_fit_selected, deviation_percent


In [4]:
peaks_calibration = [1.098, 10.5436]
print(f'Calibrating on {ga_names[-1]}, with peaks {peaks_calibration}')
dispersion, offset = calibrate_spectrum(ga[-1], peaks_calibration, plot_fit=False)
print(f'dispersion: {dispersion}, offset: {offset}')


Calibrating on GaAs_30kV, with peaks [1.098, 10.5436]
The calibration factor is: 0.0100300 keV/channel, with 21.127 channels zero offset
dispersion: 0.010030009470672611, offset: 21.1267053826366


In [5]:
# figure out how good the calibration is on other peaks
def calibration_check_on_peaks(spectrum, elements, lines, vacc, dispersion, offset):
    """
    Checks the calibration on a spectrum.
    Quantifies this by the deviation of the peak positions from the expected positions.

    Parameters
    ----------
    spectrum : array of floats
        intensity values
    elements : array of strings
        elements to check
    lines : array of strings
        lines to check
    vacc : int
        acceleration voltage
    dispersion : float
        dispersion of the calibration
    offset : float
        offset of the calibration in channels
    
    Returns
    -------
    deviation : array of floats
        deviation of the peak positions from the expected positions
    peaks_fit : array of floats
        peak positions of the fit, only the selected peaks
    peaks_energy : array of floats
        expected peak positions, from HyperSpy
    peaks_names : array of strings
        names of the peaks
    """    
    peaks_energy = []
    peaks_names = []
    for e in elements:
        for l in lines:
            try:
                energy = float(hs.material.elements[e].Atomic_properties['Xray_lines'][l]['energy (keV)'])
                if energy < vacc:
                    peaks_energy.append(energy)
                    peaks_names.append(f'{e} {l}')
                else:
                    print(f'{e} {l} at {energy} is above E_0 = {vacc} kV')
                if energy > vacc / 2:
                    print(f'Warning: {e:<2} {l:<2} at {energy:<8} have overvoltage below 2')
            except KeyError:
                pass
    print(f'Checking peaks: {peaks_names} at {peaks_energy}, with Vacc = {vacc} kV')

    channels = np.arange(len(spectrum))
    # keV = channel_to_kev(channels, dispersion=dispersion, offset=offset)
    fit_vals, _ = make_fit(spectrum, channels, deg=12)
    fit_peaks = fit_vals[12 + 2::3]
    fit_peaks_kev = channel_to_kev(fit_peaks, dispersion=dispersion, offset=offset)

    dev, peaks_fit_selected, dev_percent = peak_deviation(peaks_energy, fit_peaks_kev)
    print(dev)
    for i in range(len(dev)):
        print(f'{peaks_names[i]:<6} {peaks_fit_selected[i]:<8.4f} deviates {dev[i]:.1f} eV from {peaks_energy[i]} ({dev_percent[i]:.3f} %)')

    return dev, peaks_fit_selected, peaks_energy, peaks_names, dev_percent

In [6]:
ga30 = calibration_check_on_peaks(ga[-1], ['As', 'Ga'], ['Ka', 'La'], 30, dispersion, offset)
ga15 = calibration_check_on_peaks(ga[-2], ['As', 'Ga'], ['Ka', 'La'], 15, dispersion, offset)
ga10 = calibration_check_on_peaks(ga[-3], ['As', 'Ga'], ['La'], 10, dispersion, offset)
ga05 = calibration_check_on_peaks(ga[0], ['As', 'Ga'], ['La'], 5, dispersion, offset)


Checking peaks: ['As Ka', 'As La', 'Ga Ka', 'Ga La'] at [10.5436, 1.2819, 9.2517, 1.098], with Vacc = 30 kV
[0.         4.72343492 2.18726621 0.        ]
As Ka  10.5436  deviates 0.0 eV from 10.5436 (0.000 %)
As La  1.2866   deviates 4.7 eV from 1.2819 (0.368 %)
Ga Ka  9.2539   deviates 2.2 eV from 9.2517 (0.024 %)
Ga La  1.0980   deviates 0.0 eV from 1.098 (0.000 %)
Checking peaks: ['As Ka', 'As La', 'Ga Ka', 'Ga La'] at [10.5436, 1.2819, 9.2517, 1.098], with Vacc = 15 kV
[-6.4326328   4.82293385  2.08849021  1.41874845]
As Ka  10.5372  deviates -6.4 eV from 10.5436 (-0.061 %)
As La  1.2867   deviates 4.8 eV from 1.2819 (0.376 %)
Ga Ka  9.2538   deviates 2.1 eV from 9.2517 (0.023 %)
Ga La  1.0994   deviates 1.4 eV from 1.098 (0.129 %)
Checking peaks: ['As La', 'Ga La'] at [1.2819, 1.098], with Vacc = 10 kV
[5.09112301 2.14722683]
As La  1.2870   deviates 5.1 eV from 1.2819 (0.397 %)
Ga La  1.1001   deviates 2.1 eV from 1.098 (0.196 %)
Checking peaks: ['As La', 'Ga La'] at [1.2819, 1.0

In [7]:
# see lines
# hs.material.elements['Fe'].Atomic_properties['Xray_lines']

In [8]:
nw30 = calibration_check_on_peaks(nw[-1], ['As', 'Ga', 'Cu', ], ['La', 'Ka',], 30, dispersion, offset)

Checking peaks: ['As La', 'As Ka', 'Ga La', 'Ga Ka', 'Cu La', 'Cu Ka'] at [1.2819, 10.5436, 1.098, 9.2517, 0.9295, 8.0478], with Vacc = 30 kV
[ 5.40676788 -1.05358603  3.49089897  2.24209655  7.96298108  3.66170416]
As La  1.2873   deviates 5.4 eV from 1.2819 (0.422 %)
As Ka  10.5425  deviates -1.1 eV from 10.5436 (-0.010 %)
Ga La  1.1015   deviates 3.5 eV from 1.098 (0.318 %)
Ga Ka  9.2539   deviates 2.2 eV from 9.2517 (0.024 %)
Cu La  0.9375   deviates 8.0 eV from 0.9295 (0.857 %)
Cu Ka  8.0515   deviates 3.7 eV from 8.0478 (0.045 %)


In [9]:
dispersion_AZ = 0.01
offset_AZ = 20
nw30_AZ = calibration_check_on_peaks(nw[-1], ['As', 'Ga', 'Cu', ], ['La', 'Ka',], 30, dispersion_AZ, offset_AZ)

Checking peaks: ['As La', 'As Ka', 'Ga La', 'Ga Ka', 'Cu La', 'Cu Ka'] at [1.2819, 10.5436, 1.098, 9.2517, 0.9295, 8.0478], with Vacc = 30 kV
[ 12.82224062 -21.32949718  11.46232691 -14.17835129  16.42517536
  -9.16096044]
As La  1.2947   deviates 12.8 eV from 1.2819 (1.000 %)
As Ka  10.5223  deviates -21.3 eV from 10.5436 (-0.202 %)
Ga La  1.1095   deviates 11.5 eV from 1.098 (1.044 %)
Ga Ka  9.2375   deviates -14.2 eV from 9.2517 (-0.153 %)
Cu La  0.9459   deviates 16.4 eV from 0.9295 (1.767 %)
Cu Ka  8.0386   deviates -9.2 eV from 8.0478 (-0.114 %)


In [10]:
# # # if I use calibration from model fit in HyperSpy, Ga Ka_Calibrated center_ is 9.2525 and not 9.2528
# HyperSpy modelfit of 30 kV: dispersion = 0.010028, offset = 0.211377  / 0.010028

dispersion_HS = 0.010028
offset_HS =  0.211377  / 0.010028
nw30_HS = calibration_check_on_peaks(nw[-1], ['As', 'Ga', 'Cu', ], ['La', 'Ka',], 30, dispersion_HS, offset_HS)

Checking peaks: ['As La', 'As Ka', 'Ga La', 'Ga Ka', 'Cu La', 'Cu Ka'] at [1.2819, 10.5436, 1.098, 9.2517, 0.9295, 8.0478], with Vacc = 30 kV
[ 5.6304629  -2.68413977  3.75182143  0.86970932  8.25676585  2.53022888]
As La  1.2875   deviates 5.6 eV from 1.2819 (0.439 %)
As Ka  10.5409  deviates -2.7 eV from 10.5436 (-0.025 %)
Ga La  1.1018   deviates 3.8 eV from 1.098 (0.342 %)
Ga Ka  9.2526   deviates 0.9 eV from 9.2517 (0.009 %)
Cu La  0.9378   deviates 8.3 eV from 0.9295 (0.888 %)
Cu Ka  8.0503   deviates 2.5 eV from 8.0478 (0.031 %)


In [11]:
# Calibrated on Mo La and Mo Ka at the bottom of this file. Got:
dispersion_mo = 0.010039892338014457
offset_mo = 21.076486916545974

nw30_Mo = calibration_check_on_peaks(nw[-1], ['As', 'Ga', 'Cu', ], ['La', 'Ka',], 30, dispersion_mo, offset_mo)

Checking peaks: ['As La', 'As Ka', 'Ga La', 'Ga Ka', 'Cu La', 'Cu Ka'] at [1.2819, 10.5436, 1.098, 9.2517, 0.9295, 8.0478], with Vacc = 30 kV
[ 7.1793776   9.83848723  5.08041878 11.86446956  9.3908793  12.09923739]
As La  1.2891   deviates 7.2 eV from 1.2819 (0.560 %)
As Ka  10.5534  deviates 9.8 eV from 10.5436 (0.093 %)
Ga La  1.1031   deviates 5.1 eV from 1.098 (0.463 %)
Ga Ka  9.2636   deviates 11.9 eV from 9.2517 (0.128 %)
Cu La  0.9389   deviates 9.4 eV from 0.9295 (1.010 %)
Cu Ka  8.0599   deviates 12.1 eV from 8.0478 (0.150 %)


In [12]:
mo30 = calibration_check_on_peaks(mo[-1], ['Mo'], ['Ka','La'], 30, dispersion, offset)
mo30_AZ = calibration_check_on_peaks(mo[-1], ['Mo'], ['Ka','La'], 30, dispersion_AZ, offset_AZ)
mo30_HS = calibration_check_on_peaks(mo[-1], ['Mo'], ['Ka','La'], 30, dispersion_HS, offset_HS)
mo30_Mo = calibration_check_on_peaks(mo[-1], ['Mo'], ['Ka','La'], 30, dispersion_mo, offset_mo)

Checking peaks: ['Mo Ka', 'Mo La'] at [17.4793, 2.2932], with Vacc = 30 kV
[-15.81438593  19.67075812]
Mo Ka  17.4635  deviates -15.8 eV from 17.4793 (-0.090 %)
Mo La  2.3129   deviates 19.7 eV from 2.2932 (0.858 %)
Checking peaks: ['Mo Ka', 'Mo La'] at [17.4793, 2.2932], with Vacc = 30 kV
[-56.79752797  24.01777589]
Mo Ka  17.4225  deviates -56.8 eV from 17.4793 (-0.325 %)
Mo La  2.3172   deviates 24.0 eV from 2.2932 (1.047 %)
Checking peaks: ['Mo Ka', 'Mo La'] at [17.4793, 2.2932], with Vacc = 30 kV
[-18.83152105  19.68898566]
Mo Ka  17.4605  deviates -18.8 eV from 17.4793 (-0.108 %)
Mo La  2.3129   deviates 19.7 eV from 2.2932 (0.859 %)
Checking peaks: ['Mo Ka', 'Mo La'] at [17.4793, 2.2932], with Vacc = 30 kV
[ 1.89709505 22.45388662]
Mo Ka  17.4812  deviates 1.9 eV from 17.4793 (0.011 %)
Mo La  2.3157   deviates 22.5 eV from 2.2932 (0.979 %)


In [13]:
si30 = calibration_check_on_peaks(si[-1], ['Si'], ['Ka'], 30, dispersion, offset)
si30_AZ = calibration_check_on_peaks(si[-1], ['Si'], ['Ka'], 30, dispersion_AZ, offset_AZ)
si30_HS = calibration_check_on_peaks(si[-1], ['Si'], ['Ka'], 30, dispersion_HS, offset_HS)
si30_Mo = calibration_check_on_peaks(si[-1], ['Si'], ['Ka'], 30, dispersion_mo, offset_mo)

Checking peaks: ['Si Ka'] at [1.7397], with Vacc = 30 kV
[-3.17428326]
Si Ka  1.7365   deviates -3.2 eV from 1.7397 (-0.182 %)
Checking peaks: ['Si Ka'] at [1.7397], with Vacc = 30 kV
[2.89714062]
Si Ka  1.7426   deviates 2.9 eV from 1.7397 (0.167 %)
Checking peaks: ['Si Ka'] at [1.7397], with Vacc = 30 kV
[-3.04058739]
Si Ka  1.7367   deviates -3.0 eV from 1.7397 (-0.175 %)
Checking peaks: ['Si Ka'] at [1.7397], with Vacc = 30 kV
[-0.95904471]
Si Ka  1.7387   deviates -1.0 eV from 1.7397 (-0.055 %)


In [14]:
# on Fe, which is really Al
al30 = calibration_check_on_peaks(fe[0], ['Al'], ['Ka'], 30, dispersion, offset)
al30_AZ = calibration_check_on_peaks(fe[0], ['Al'], ['Ka'], 30, dispersion_AZ, offset_AZ)
al30_HS = calibration_check_on_peaks(fe[0], ['Al'], ['Ka'], 30, dispersion_HS, offset_HS)
al30_Mo = calibration_check_on_peaks(fe[0], ['Al'], ['Ka'], 30, dispersion_mo, offset_mo)

Checking peaks: ['Al Ka'] at [1.4865], with Vacc = 30 kV
[-3.85615855]
Al Ka  1.4826   deviates -3.9 eV from 1.4865 (-0.259 %)
Checking peaks: ['Al Ka'] at [1.4865], with Vacc = 30 kV
[2.97487186]
Al Ka  1.4895   deviates 3.0 eV from 1.4865 (0.200 %)
Checking peaks: ['Al Ka'] at [1.4865], with Vacc = 30 kV
[-3.6715985]
Al Ka  1.4828   deviates -3.7 eV from 1.4865 (-0.247 %)
Checking peaks: ['Al Ka'] at [1.4865], with Vacc = 30 kV
[-1.89107738]
Al Ka  1.4846   deviates -1.9 eV from 1.4865 (-0.127 %)


In [15]:
# on Cu, which is mostly C
c30 = calibration_check_on_peaks(cu[-1], ['Cu', 'C'], ['Ka'], 30, dispersion, offset)
c30_AZ = calibration_check_on_peaks(cu[-1], ['Cu', 'C'], ['Ka'], 30, dispersion_AZ, offset_AZ)
c30_HS = calibration_check_on_peaks(cu[-1], ['Cu', 'C'], ['Ka'], 30, dispersion_HS, offset_HS)
c30_Mo = calibration_check_on_peaks(cu[-1], ['Cu', 'C'], ['Ka'], 30, dispersion_mo, offset_mo)

Checking peaks: ['Cu Ka', 'C Ka'] at [8.0478, 0.2774], with Vacc = 30 kV
[  3.49849352 -18.6903289 ]
Cu Ka  8.0513   deviates 3.5 eV from 8.0478 (0.043 %)
C Ka   0.2587   deviates -18.7 eV from 0.2774 (-6.738 %)
Checking peaks: ['Cu Ka', 'C Ka'] at [8.0478, 0.2774], with Vacc = 30 kV
[-9.32368275 -8.19732621]
Cu Ka  8.0385   deviates -9.3 eV from 8.0478 (-0.116 %)
C Ka   0.2692   deviates -8.2 eV from 0.2774 (-2.955 %)
Checking peaks: ['Cu Ka', 'C Ka'] at [8.0478, 0.2774], with Vacc = 30 kV
[  2.36705093 -18.26055873]
Cu Ka  8.0502   deviates 2.4 eV from 8.0478 (0.029 %)
C Ka   0.2591   deviates -18.3 eV from 0.2774 (-6.583 %)
Checking peaks: ['Cu Ka', 'C Ka'] at [8.0478, 0.2774], with Vacc = 30 kV
[ 11.93586594 -17.93122655]
Cu Ka  8.0597   deviates 11.9 eV from 8.0478 (0.148 %)
C Ka   0.2595   deviates -17.9 eV from 0.2774 (-6.464 %)


In [16]:
nw30_HS

(array([ 5.6304629 , -2.68413977,  3.75182143,  0.86970932,  8.25676585,
         2.53022888]),
 array([ 1.28753046, 10.54091586,  1.10175182,  9.25256971,  0.93775677,
         8.05033023]),
 [1.2819, 10.5436, 1.098, 9.2517, 0.9295, 8.0478],
 ['As La', 'As Ka', 'Ga La', 'Ga Ka', 'Cu La', 'Cu Ka'],
 array([ 0.43922793, -0.02545753,  0.34169594,  0.00940054,  0.88830187,
         0.03144001]))

In [17]:
hyper = [nw30_HS, mo30_HS, si30_HS, al30_HS, c30_HS]
aztec = [nw30_AZ, mo30_AZ, si30_AZ, al30_AZ, c30_AZ]
mycalib = [nw30, mo30, si30, al30, c30]
mo_calib = [nw30_Mo, mo30_Mo, si30_Mo, al30_Mo, c30_Mo]

alpha = '$\\alpha$'
sp = ' & '

header = f'{"Peak":<15} & {"Theoretical [keV]":<20} & {"AZ calib. [eV]":<15} & {"HS calib. [eV]":<15} &'
header += f'{"Calib. Ga L alpha *& As K alpha [eV]":<15} & {"Calib. Mo L alpha *& Mo K alpha [eV]":<15} \\\\'
print(header)
print('\hline')
for i in range(len(hyper)):
    for j in range(len(hyper[i][0])):
        line = ''
        line += f'{hyper[i][3][j][:-1]+alpha:<15} & {hyper[i][2][j]:<20} &'  # peak and theoretical
        line += f'  {aztec[i][0][j]:>6.1f},\,\,\, {aztec[i][4][j]:>6.1f}\% {sp:<10}'
        line += f'  {hyper[i][0][j]:>6.1f},\,\,\, {hyper[i][4][j]:>6.1f}\% {sp:<10}'
        line += f'  {mycalib[i][0][j]:>6.1f},\,\,\, {mycalib[i][4][j]:>6.1f}\% {sp:<10}'
        line += f'  {mo_calib[i][0][j]:>6.1f},\,\,\, {mo_calib[i][4][j]:>6.1f}\% \\\\'
        print(line) 


def rms(error):
    return np.sqrt(np.mean(error**2))


line = f'\hline\nRMS error [eV]  & {sp:>23}  ' 
line += f'  {rms(aztec[0][0]):>6.2f} {sp:<15} {rms(hyper[0][0]):>6.2f} {sp:<15}' 
line += f'  {rms(mycalib[0][0]):>6.2f} {sp:<15}  {rms(mo_calib[0][0]):>6.2f} '
print(line)

Peak            & Theoretical [keV]    & AZ calib. [eV]  & HS calib. [eV]  &Calib. Ga L alpha *& As K alpha [eV] & Calib. Mo L alpha *& Mo K alpha [eV] \\
\hline
As L$\alpha$    & 1.2819               &    12.8,\,\,\,    1.0\%  &             5.6,\,\,\,    0.4\%  &             5.4,\,\,\,    0.4\%  &             7.2,\,\,\,    0.6\% \\
As K$\alpha$    & 10.5436              &   -21.3,\,\,\,   -0.2\%  &            -2.7,\,\,\,   -0.0\%  &            -1.1,\,\,\,   -0.0\%  &             9.8,\,\,\,    0.1\% \\
Ga L$\alpha$    & 1.098                &    11.5,\,\,\,    1.0\%  &             3.8,\,\,\,    0.3\%  &             3.5,\,\,\,    0.3\%  &             5.1,\,\,\,    0.5\% \\
Ga K$\alpha$    & 9.2517               &   -14.2,\,\,\,   -0.2\%  &             0.9,\,\,\,    0.0\%  &             2.2,\,\,\,    0.0\%  &            11.9,\,\,\,    0.1\% \\
Cu L$\alpha$    & 0.9295               &    16.4,\,\,\,    1.8\%  &             8.3,\,\,\,    0.9\%  &             8.0,\,\,\,    0.9\%  &         

In [178]:
mo_calib[0][1]

array([ 1.29, 10.55,  1.1 ,  9.26,  0.94,  8.06])

In [162]:
# RMS error on the calibration



print('RMS error, sum of error, sum of percent error')
print('AZTEC: ', rms(aztec[0][0]), np.sum(np.abs(aztec[0][0])), np.sum(np.abs(aztec[0][4])))
print('HyperSpy: ', rms(hyper[0][0]), np.sum(np.abs(hyper[0][0])), np.sum(np.abs(hyper[0][4])))
print('My calib: ', rms(mycalib[0][0]), np.sum(np.abs(mycalib[0][0])), np.sum(np.abs(mycalib[0][4])))
print('Mo calib: ', rms(mo_calib[0][0]), np.sum(np.abs(mo_calib[0][0])), np.sum(np.abs(mo_calib[0][4])))

# set np print options
np.set_printoptions(precision=2, suppress=True)


RMS error, sum of error, sum of percent error
AZTEC:  14.75057891485451 85.37855180628307 4.280659727534262
HyperSpy:  4.624491279859571 23.7231281466278 1.7355238111443445
My calib:  4.552905665831109 23.818034665822108 1.6761318188330132
Mo calib:  9.569881106603354 55.45286987669606 2.404965758900323


In [39]:
# fitting Mo 30 kV manually, since it has the double peak

# will only fit on a small window to find the center of the tow peaks in the double peak
# fitting from 200 to 300, peaks at 220, 250 and 260

# also need to fit the peak at 1760, with window from 1700 to 1850

# I only need the excact center of the peaks, not the full fit
start1 = 200
end1 = 300
peaks_channels1 = [220, 250, 260]

start2 = 1700
end2 = 1850
peaks_channels2 = [1760]

# fitting on the first peaks
mo30 = mo[-1]

def three_gaussians(x, a1, c1, w1, a2, c2, w2, a3, c3, w3):
    return a1 * np.exp(-((x - c1) / w1) ** 2) + a2 * np.exp(-((x - c2) / w2) ** 2) + a3 * np.exp(-((x - c3) / w3) ** 2)

def gaussian(x, a, c, w):
    return a * np.exp(-((x - c) / w) ** 2)

In [40]:
from scipy.optimize import curve_fit
channels = np.arange(len(mo30))

fit_vals_mo_la, _ = curve_fit(three_gaussians, channels[start1:end1], mo30[start1:end1], p0=[1, 220, 1, 1, 250, 1, 1, 260, 1])
fit_vals_mo_la[1::3]

array([244.57562427, 249.48531072, 259.69752324])

In [41]:
fit = three_gaussians(channels[start1:end1], *fit_vals_mo_la)

fig = go.Figure()
fig.add_trace(go.Scatter(x=channels[start1:end1], y=mo30[start1:end1], name='data'))
fig.add_trace(go.Scatter(x=channels[start1:end1], y=fit, name='fit'))
# add the center of the two last peaks
fig.add_trace(go.Scatter(x=[fit_vals_mo_la[7], fit_vals_mo_la[4]], y=[0, 0], mode='markers', name='centers'))
# add the single gaussians
for i in range(3):
    fit = gaussian(channels[start1:end1], fit_vals_mo_la[i*3], fit_vals_mo_la[i*3+1], fit_vals_mo_la[i*3+2])
    fig.add_trace(go.Scatter(x=channels[start1:end1], y=fit, name='fit'))

fig.update_layout(title=f'Mo La have center at channel {fit_vals_mo_la[4]}', xaxis_title='Channel', yaxis_title='Counts')
# fig.show()

'mo'

'mo'

In [42]:
# fit Mo Ka with one gaussian
fit_vals_mo_ka, _ = curve_fit(gaussian, channels[start2:end2], mo30[start2:end2], p0=[1, 1760, 1])
print(fit_vals_mo_ka[1])
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=channels[start2:end2], y=mo30[start2:end2], name='data'))
fig2.add_trace(go.Scatter(x=channels[start2:end2], y=gaussian(channels[start2:end2], *fit_vals_mo_ka), name='fit'))
fig2.update_layout(title=f'Mo Ka have center at channel {fit_vals_mo_ka[1]}', xaxis_title='Channel', yaxis_title='Counts')
# fig2.show()

'mo'

1762.0612914863525


'mo'

In [43]:
# the dispersion with Mo La and Mo Ka

mo_ka = 17.4793 # keV
mo_la = 2.2932 # keV
dispersion_mo_ka_la = (mo_ka - mo_la) / (fit_vals_mo_ka[1] - fit_vals_mo_la[4])
offset_keV_mo_ka_la = mo_la - dispersion_mo_ka_la * fit_vals_mo_la[4]
offset_mo_ka_la = fit_vals_mo_la[4] - mo_la / dispersion_mo_ka_la
print(f'dispersion: {dispersion_mo_ka_la}, offset: {offset_mo_ka_la} channels, {offset_keV_mo_ka_la} keV')

dispersion_mo = 0.010039892338014457
offset_mo = 21.076486916545974

dispersion: 0.010039892338014457, offset: 21.076486916545974 channels, -0.21160565950569188 keV


# Mo calibration
## dispersion: 0.010039892338014457, offset: 21.076486916545974 channels